In [1]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

c:\Users\alam\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

#### tweet1

In [2]:
data1 = pd.read_excel('dataset\SentimentIbu_kota_pindah_new.xlsx')
data2 = pd.read_excel('dataset\SentimentIPrmindahan_bu_Kota_Jakarta.xlsx') 
data3 = pd.read_excel('dataset\SentimentPemindahan_Ibu_Kota_new.xlsx')

data_tweet = pd.concat([data1,data2,data3],ignore_index=True)
data_tweet = data_tweet['text']

In [3]:
data_tweet = data_tweet.drop_duplicates(ignore_index=True)
data_tweet

0       @detikcom Maksudnya secara gak langsung mau bi...
1       @san_ozil saya turut mendukung proses pemindah...
2       Kolaborasi menjadi salah satu kunci bagi pemer...
3       Sch! GUYS AKU KETERIMA DI ILKOM UNPAD 😭😭😭😭😭\n\...
4       @abangbelneg ayo kawal proses pemindahan Ibuko...
                              ...                        
2096    @Ranisor Yg bisa atasin banjir pas jadi presid...
2097     saya turut mendukung proses pemindahan Ibukot...
2098     Trump menangguhkan pemindahan ibukota Israel ...
2099    Perpindahan Ibu Kota Negara (IKN) \nIndonesia ...
2100    IKN idaman\n\nPasal 24\n\n(1) Pendanaan untuk ...
Name: text, Length: 2101, dtype: object

#### tweet2

In [4]:
tweet2_1 = pd.read_csv('dataset\ikn.csv',delimiter=';')
tweet2_2 = pd.read_csv('dataset\masalah_ikn.csv',delimiter=';')
tweet2_3 = pd.read_csv('dataset\masalah_ikn.old.csv',delimiter=';')
tweet2_4 = pd.read_csv('dataset\Pembangunan_ikn.csv',delimiter=';')
tweet2_5 = pd.read_csv('dataset\postif_ikn.csv',delimiter=';')

tweet2_all = pd.concat([tweet2_1,tweet2_2,tweet2_4,tweet2_4,tweet2_5],ignore_index=True)
tweet2_all = tweet2_all['full_text']
tweet2_all = tweet2_all.rename('text')
tweet2_all

0       @kompascom Dari sabang sampe merauke dah bnyk ...
1       Nyetir tengah malem di IKN is another level of...
2       framingnya mulai muncul. IKN jadi kata kunci. ...
3       IKN masih proyek guys, buat yg ASN santai-sant...
4       @mkusumawijaya coba ty amdal ikn. dibuat sesua...
                              ...                        
2317    Wih keren nih guys PLN bangun PLTS IKN 50 MW l...
2318    @jokowi Rehabilitasi dan perbaikan sekolah den...
2319    Selain pembangunan, Dampak positif dari IKN ad...
2320    IKN berdampak positif utk pertumbuhan ekonomi ...
2321    @GenZNusantara @ikn_id @_Umamlik @CerahID @Ghu...
Name: text, Length: 2322, dtype: object

In [5]:
tweet2_all = tweet2_all.drop_duplicates(ignore_index=True)


In [6]:
tweet2_all 

0       @kompascom Dari sabang sampe merauke dah bnyk ...
1       Nyetir tengah malem di IKN is another level of...
2       framingnya mulai muncul. IKN jadi kata kunci. ...
3       IKN masih proyek guys, buat yg ASN santai-sant...
4       @mkusumawijaya coba ty amdal ikn. dibuat sesua...
                              ...                        
1536    Wih keren nih guys PLN bangun PLTS IKN 50 MW l...
1537    @jokowi Rehabilitasi dan perbaikan sekolah den...
1538    Selain pembangunan, Dampak positif dari IKN ad...
1539    IKN berdampak positif utk pertumbuhan ekonomi ...
1540    @GenZNusantara @ikn_id @_Umamlik @CerahID @Ghu...
Name: text, Length: 1541, dtype: object

#### test youtube

In [49]:
# data4 = pd.read_excel('youtube_scraping\Youtube_short1.xlsx')
# data5 = pd.read_excel('youtube_scraping\Youtube_video1.xlsx')
# data_youtube = pd.concat([data4,data5])

# data_youtube = data_youtube.rename(columns={"sentimen":"text"})


In [50]:
# data_youtube = data_youtube["text"].dropna()
# data_youtube

0       RAKYAT REMPANG MENANGIS MEREKA MEMBUAT SYUKURAN 
1      bukannya lu sendiri yg bilang kalo ras Jawa it...
2       @rizalwahyu2152 yang bilang Jawa itu babu siapa 
3       @rizalwahyu2152 jawa mending merdeka sendiri ...
4       @user-iw6ow2bk5y  \nYa itu lebih bagus lagi.....
                             ...                        
219    Kota paling Hijau, apakah hutan sebelumnya tid...
220    Ikn tidak di butuhkan karena negara kita sedan...
221    IKN akan memberatkan APBN yg seharusnya untuk ...
222    Senjata hatersnya yg kurang literasi nggak jau...
223    Saya yakin, pak jokowi lengser IKN akan menjad...
Name: text, Length: 224, dtype: object

#### combine all data

In [7]:
data = pd.concat([data_tweet,tweet2_all])
data.info()


<class 'pandas.core.series.Series'>
Index: 3642 entries, 0 to 1540
Series name: text
Non-Null Count  Dtype 
--------------  ----- 
3642 non-null   object
dtypes: object(1)
memory usage: 56.9+ KB


In [8]:
data.count()

3642

In [9]:
data.to_excel('data_sentiment1.xlsx')

# preprocessing

### cleaning + casefolding

In [54]:
# casefolding + cleaning 
# function 
def clean_tweet(teks):
    return ''.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+\/\/\S+)","",teks))
def clean_tweet(teks):
    return re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+\/\/\S+)","",teks)
def remove_number(text):
    return  re.sub(r"\d+", "", text)

#use function
clean = []
for i in data:
    clean.append(clean_tweet(i))

In [55]:
clean_dat = pd.DataFrame(clean, columns=['teks'])
clean_dat['teks'] = clean_dat['teks'].str.lower()
clean_dat['teks'] = clean_dat['teks'].apply(remove_number)

In [56]:
clean_dat

,teks
0,maksudnya secara gak langsung mau bikin pembe...
1,ozil saya turut mendukung proses pemindahan ib...
2,kolaborasi menjadi salah satu kunci bagi pemer...
3,sch guys aku keterima di ilkom unpad trivia s...
4,ayo kawal proses pemindahan ibukota ibukotane...
...,...
2320,kota paling hijau apakah hutan sebelumnya tida...
2321,ikn tidak di butuhkan karena negara kita sedan...
2322,ikn akan memberatkan apbn yg seharusnya untuk ...
2323,senjata hatersnya yg kurang literasi nggak jau...


### tokenizing

In [57]:
#function
def word_tokenize_wrapper(text):
    return word_tokenize(text)

In [58]:
clean_dat['teks'] = clean_dat['teks'].apply(word_tokenize_wrapper)



In [59]:
clean_dat.head()

,teks
0,"[maksudnya, secara, gak, langsung, mau, bikin,..."
1,"[ozil, saya, turut, mendukung, proses, peminda..."
2,"[kolaborasi, menjadi, salah, satu, kunci, bagi..."
3,"[sch, guys, aku, keterima, di, ilkom, unpad, t..."
4,"[ayo, kawal, proses, pemindahan, ibukota, ibuk..."


### normalisasi

In [57]:
norm_dat = pd.read_csv('kamus_normalisasi.csv',delimiter=',')
norm_dat

norm_dat2 = pd.read_csv('kbba.txt', delimiter='\t',encoding='latin-1', header=None)
norm_dat2 = norm_dat2.rename(columns={ 0:'slang', 1:'formal'})


In [58]:
norm_dat = norm_dat.drop(['context','In-dictionary','category1','category2','category3'],axis=1)

In [59]:
norm_dat = pd.concat([norm_dat,norm_dat2],ignore_index=True)


In [60]:
norm_dat = norm_dat.drop_duplicates()
norm_dat

,slang,formal
0,woww,wow
1,aminn,amin
2,met,selamat
3,netaas,menetas
4,keberpa,keberapa
...,...,...
16318,ahokncc,ahok
16319,almaidahyg,almaidah
16321,benarjujur,benar
16322,benarjujur,jujur


In [64]:
normalizad_word_dict = {}

for index, row in norm_dat.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

In [65]:
normalizad_word_dict

{'woww': 'wow',
 'aminn': 'amin',
 'met': 'selamat',
 'netaas': 'menetas',
 'keberpa': 'keberapa',
 'eeeehhhh': 'eh',
 'kata2nyaaa': 'kata-katanya',
 'hallo': 'halo',
 'kaka': 'kakak',
 'ka': 'kak',
 'daah': 'dah',
 'aaaaahhhh': 'ah',
 'yaa': 'ya',
 'smga': 'semoga',
 'slalu': 'selalu',
 'amiin': 'amin',
 'kk': 'kakak',
 'trus': 'terus',
 'sii': 'sih',
 'nyenengin': 'menyenangkan',
 'bgt': 'banget',
 'gemess': 'gemas',
 'akuuu': 'aku',
 'jgn': 'jangan',
 'udah': 'sudah',
 'gitu': 'begitu',
 'aja': 'saja',
 'gemesiin': 'menggemaskan',
 'menyenangkn': 'menyenangkan',
 'rb': 'ribu',
 'akau': 'aku',
 'saranin': 'menyarankan',
 'nemuin': 'menemukan',
 'yah': 'ya',
 'mnis2nya': 'manis-manisnya',
 'gtu': 'begitu',
 'tu': 'itu',
 'idolaa': 'idola',
 'bangett': 'banget',
 'kaakk': 'kak',
 'baaiikkk': 'baik',
 'telaatt': 'telat',
 'kaak': 'kak',
 'yg': 'yang',
 'lg': 'lagi',
 'skarang': 'sekarang',
 'sempet': 'sempat',
 'karna': 'karena',
 'ngga': 'enggak',
 'mantep': 'mantap',
 'loohhh': 'loh',

In [66]:
# function
def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

clean_dat['teks'] = clean_dat['teks'].apply(normalized_term)

In [67]:
clean_dat

,teks
0,"[maksudnya, secara, enggak, langsung, mau, bik..."
1,"[ozil, saya, turut, mendukung, proses, peminda..."
2,"[kolaborasi, menjadi, salah, satu, kunci, bagi..."
3,"[sch, guys, aku, keterima, di, ilkom, universi..."
4,"[ayo, kawal, proses, pemindahan, ibu kota, ibu..."
...,...
2320,"[kota, paling, hijau, apakah, hutan, sebelumny..."
2321,"[ikn, tidak, di, butuhkan, karena, negara, kit..."
2322,"[ikn, akan, memberatkan, anggaran pendapatan d..."
2323,"[senjata, hatersnya, yang, kurang, literasi, e..."


### stopword removal

In [68]:
list_stopword = stopwords.words('indonesian')
sastrawi_stopwords = StopWordRemoverFactory().get_stop_words()

# bila ada kata yang mau di masukan stopword secara manual 
# list_stopword.extend(['...'])    

txt_stopword = pd.read_csv("id.stopwords.02.01.2016.txt", names= ["stopwords"], header = None)

# menambah stopword berdasarkan kamus 
list_stopword.extend(txt_stopword["stopwords"][0].split(' '))
list_stopword.extend(sastrawi_stopwords)

In [69]:
list_stopword = set(list_stopword)

# function
def stopwords_removal(words):
    return [word for word in words if word not in list_stopword]

clean_dat['teks'] = clean_dat['teks'].apply(stopwords_removal)

In [70]:
clean_dat.head()

,teks
0,"[maksudnya, langsung, bikin, pembenaran, pemin..."
1,"[ozil, mendukung, proses, pemindahan, ibu kota..."
2,"[kolaborasi, salah, kunci, pemerataan, investa..."
3,"[sch, guys, keterima, ilkom, universitas neger..."
4,"[ayo, kawal, proses, pemindahan, ibu kota, ibu..."


### Stemming

In [71]:
clean_dat

,teks
0,"[maksudnya, langsung, bikin, pembenaran, pemin..."
1,"[ozil, mendukung, proses, pemindahan, ibu kota..."
2,"[kolaborasi, salah, kunci, pemerataan, investa..."
3,"[sch, guys, keterima, ilkom, universitas neger..."
4,"[ayo, kawal, proses, pemindahan, ibu kota, ibu..."
...,...
2320,"[kota, hijau, hutan, hijau]"
2321,"[ikn, butuhkan, negara]"
2322,"[ikn, memberatkan, anggaran pendapatan dan bel..."
2323,"[senjata, hatersnya, literasi, mobil, ghoib, e..."


In [72]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stem test
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)

print(output)

ekonomi indonesia sedang dalam tumbuh yang bangga


In [73]:
import swifter
# funciton 
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in clean_dat['teks']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])

# fucntion for dataframe 
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

clean_dat['teks'] = clean_dat['teks'].swifter.apply(get_stemmed_term)

7678
------------------------
maksudnya : maksud
langsung : langsung
bikin : bikin
pembenaran : benar
pemindahan : pindah
ibu kota : ibu kota
ikn : ikn
halus : halus
acara : acara
macet : macet
dan lain lain : dan lain lain
basi : basi
tong : tong
sidah : sidah
presiden : presiden
gampang : gampang
atasi : atas
lo : lo
lagu : lagu
putar : putar
ozil : ozil
mendukung : dukung
proses : proses
negara : negara
pindahibukota : pindahibukota
kolaborasi : kolaborasi
salah : salah
kunci : kunci
pemerataan : perata
investasi : investasi
indonesia : indonesia
upaya : upaya
pemerintah : perintah
mewujudkan : wujud
citacita : citacita
menggenjot : genjot
investor : investor
daerah : daerah
sch : sch
guys : guys
keterima : terima
ilkom : ilkom
universitas negeri padjadjaran : universitas negeri padjadjaran
trivia : trivia
stasiun : stasiun
kak : kak
bandung : bandung
dirancang : rancang
persiapan : siap
jakarta : jakarta
httpstcomtekdspb : httpstcomtekdspb
ayo : ayo
kawal : kawal
ibukotanegara : ib

Pandas Apply: 100%|██████████| 2325/2325 [00:00<00:00, 312837.06it/s]


In [74]:
clean_dat

,teks
0,"[maksud, langsung, bikin, benar, pindah, ibu k..."
1,"[ozil, dukung, proses, pindah, ibu kota, negar..."
2,"[kolaborasi, salah, kunci, perata, investasi, ..."
3,"[sch, guys, terima, ilkom, universitas negeri ..."
4,"[ayo, kawal, proses, pindah, ibu kota, ibukota..."
...,...
2320,"[kota, hijau, hutan, hijau]"
2321,"[ikn, butuh, negara]"
2322,"[ikn, berat, anggar dapat dan belanja negara, ..."
2323,"[senjata, hatersnya, literasi, mobil, ghoib, e..."


### Labeling

In [75]:
# import from hugging face test
from transformers import pipeline

pretrained_name = "w11wo/indonesian-roberta-base-sentiment-classifier"

nlp = pipeline(
    "sentiment-analysis",
    model=pretrained_name,
    tokenizer=pretrained_name
)

nlp("Jangan sampai saya telpon bos saya ya!")

[{'label': 'negative', 'score': 0.9987142086029053}]

In [76]:
hasil = nlp("Jangan sampai saya telpon bos saya ya!")

In [77]:
print(hasil[0]['label'])

negative


In [78]:
import ast

# Menggabungkan kembali kata - kata ke kalimat normal
def gabung_text_list(texts):
    return ' '.join(texts)

# Eksekusi function gabung_text_list
clean_dat['kalimat'] = clean_dat['teks'].apply(gabung_text_list)

In [79]:
clean_dat

,teks,kalimat
0,"[maksud, langsung, bikin, benar, pindah, ibu k...",maksud langsung bikin benar pindah ibu kota ik...
1,"[ozil, dukung, proses, pindah, ibu kota, negar...",ozil dukung proses pindah ibu kota negara pind...
2,"[kolaborasi, salah, kunci, perata, investasi, ...",kolaborasi salah kunci perata investasi indone...
3,"[sch, guys, terima, ilkom, universitas negeri ...",sch guys terima ilkom universitas negeri padja...
4,"[ayo, kawal, proses, pindah, ibu kota, ibukota...",ayo kawal proses pindah ibu kota ibukotanegara
...,...,...
2320,"[kota, hijau, hutan, hijau]",kota hijau hutan hijau
2321,"[ikn, butuh, negara]",ikn butuh negara
2322,"[ikn, berat, anggar dapat dan belanja negara, ...",ikn berat anggar dapat dan belanja negara pera...
2323,"[senjata, hatersnya, literasi, mobil, ghoib, e...",senjata hatersnya literasi mobil ghoib esemka ...


In [80]:
def labeling (teks):
    hasil = nlp(teks)
    return hasil[0]['label']

In [83]:
clean_dat['sentimen'] = clean_dat['kalimat'].apply(labeling)

In [84]:
print(clean_dat['sentimen'].value_counts())

sentimen
neutral     1416
positive     517
negative     391
Name: count, dtype: int64


### Balancing

In [117]:
data_x = clean_dat['sentimen'], clean_dat['teks']

In [118]:
import numpy
x,y = clean_dat.iloc[:,1], clean_dat.iloc[:,2]

x = numpy.array(x)

In [121]:
x = x.reshape(-1,1)
x.shape

(2324, 1)

In [122]:
# under sampling penerapan 
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(x, y)

In [ ]:
# over sampling penerapan 
from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = ADASYN().fit_resample(x, y)

In [90]:
print(y_resampled.value_counts())

sentimen
negative    391
neutral     391
positive    391
Name: count, dtype: int64


### pembobotan

In [91]:
x_data = X_resampled.flatten()

In [109]:
# feature ekstraction with tf idf
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=1245)
data_features = vectorizer.fit_transform(x_data)
data_features = data_features.toarray()

data_features

array([[0.        , 0.26821283, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31975531, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [108]:
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(max_features=1245)
# data_features = vectorizer.fit_transform(x_data)
# data_features_CountV = data_features.toarray()

# data_features_CountV

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [94]:
# encode label
from sklearn.preprocessing import LabelEncoder

encode = LabelEncoder()

y_encode = encode.fit_transform(y_resampled)

### apply to xgboost

In [106]:
# this is try inbalance data  using xgboost 

data_features = vectorizer.fit_transform(x.flatten())
data_features = data_features.toarray()

Y_Encode = encode.fit_transform(y)

In [128]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_features, y_encode, test_size=0.2, random_state=42, shuffle=True)

In [129]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report


xgb_model = XGBClassifier(max_depth=6, max_delta_step=5).fit(X_train,y_train)
predict = xgb_model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predict)

In [130]:
print(accuracy)

0.6085106382978723


### apply to xgboost hyper parameter

In [98]:

from xgboost import XGBClassifier
estimator = XGBClassifier(
    objective= 'multi:softmax',
    num_class = 3,
    nthread=4,
    seed=42
)

In [99]:
# xgboost parameter 1
# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0, 0.5, 1, 1.5, 2],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 6, 10, 20]
#         }

In [100]:
# xgboost parameter 2
params = {
    'learning_rate': [0.005, 0.1,0.3,0.5],
    'subsample' : [0.5,0.8,1],
    'max_leaves' :[0,3,5,7,10],
    'max_depth' : [6, 10, 20,50],
    'gamma' :[0,0.02,0.5]
        }

In [ ]:
# xgboost parameter 2 (with balancing data)
params = {
    'learning_rate': [0.005, 0.1,0.3,0.5],
    'subsample' : [0.5,0.8,1],
    'max_leaves' :[0,3,5,7,10],
    'max_depth' : [6, 10, 20,50],
    'gamma' :[0,0.02,0.5]
        }

In [101]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=params,
    scoring = 'f1_micro',
    n_jobs = 10,
    cv = 10,
    verbose=True
)

In [102]:
grid_search.fit(data_features,y_encode)

Fitting 10 folds for each of 720 candidates, totalling 7200 fits


KeyboardInterrupt: 

In [ ]:
print(grid_search.best_score_)

0.6072903225806452


#### if have test dataset

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
predict = grid_search.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predict)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predict))